In [1]:
!pip install beautifulsoup4 requests
!pip install schedule
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [2]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import schedule
import pandas as pd
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import spacy
import re

import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from textblob import TextBlob
import scipy.stats as stats
from datetime import datetime

In [3]:
pagesToGet= 1
upperframe=[]  
submeu_li = []
n_topics = 50

nlp=spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

In [4]:
#Get Soup
def get_soup(url1):

  page=""
  #an exception might be thrown, so the code should be in a try-except block
  try:
      #use the browser to get the url. This is suspicious command that might blow up.
      page=requests.get(url1)                             # this might throw an exception if something goes wrong.

  except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url1)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
  
  soup=BeautifulSoup(page.text,'html.parser')
  return soup

In [5]:
#Fetch news
def get_news(): 
  url = "https://www.npr.org/sections/news/" 
  frame =[]
  soup = get_soup(url)
  links=soup.find_all(class_="item")
  
  for j in links:
    St = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
    La = j.find("p").text.strip()
    if(len(La) > 10 and len(St) > 10):
      frame.append((St + " " +La))
    
  submeu=soup.find(class_="animated")#.find_all("li")

  for a in submeu.find_all('a', href=True):
    url = "https://www.npr.org" + a['href']
    soup = get_soup(url)
    links=soup.find_all(class_="item")
    for j in links:
      Statement = j.find("div",attrs={'class':'item-info'}).find("h2").text.strip()
      Label = j.find("p").text.strip()
      if(len(Label) > 5 and len(Statement) > 5):
        frame.append((Statement+ " " +Label))

  frame = [re.sub('\n','', f) for f in frame]
  frame = [re.sub(' +', ' ', f) for f in frame]
  frame = list(set(frame))

  return frame

In [6]:
#remove stopwords
def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in all_stopwords]
    return (" ").join(tokens_filtered)

In [7]:
#Clean data
def data_cleaning(frame):
  en_core = spacy.load('en_core_web_sm')
  #print("++++++++", frame)
  lemma_frame_ls = []
  frame = [f.lower() for f in frame]
  frame = [re.sub('hide caption','', word) for word in frame]
  frame = [re.sub('getty images','', word) for word in frame]
  frame = [re.sub('[^A-Za-z ]+', '', word) for word in frame]
  #print("--------------", frame)
  frame =[remove_mystopwords(text) for text in frame]
  frame = [re.sub(' +', ' ', word) for word in frame]

  df = pd.DataFrame(frame, columns=['data'])
  df["data_cleanned"] = df['data'].apply(lambda x: " ".join([y.lemma_ for y in en_core(x)]))

  return df

In [8]:
# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [9]:
# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

In [10]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
      try:
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        if not isinstance(temp_vector_sum, int):
          temp_vector_sum = temp_vector_sum.toarray()
          top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
          top_word_indices.append(top_n_word_indices) 
      except:
        continue  
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [11]:
#Implement LSA
def impl_lsa():
  reindexed_data = data_cleaning(get_news())
  reindexed_data = reindexed_data['data_cleanned']
  count_vectorizer = CountVectorizer(stop_words='english', max_features=40000)

  print('Headline before vectorization: {}'.format(reindexed_data[123]))

  document_term_matrix = count_vectorizer.fit_transform(reindexed_data)

  n_topics = 20
  lsa_model = TruncatedSVD(n_components=n_topics)
  lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)

  lsa_keys = get_keys(lsa_topic_matrix)
  lsa_categories, lsa_counts = keys_to_counts(lsa_keys)
  top_3_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, count_vectorizer)
  #for t in range(n_topics):
      #print(top_3_words_lda[t])
  return top_3_words_lsa

In [12]:
#Save Topic in CSV file
def save_tiopic():
  top_3_words_lsa = impl_lsa()
  now = datetime.now()
  current_time = now.strftime("%H_%M_%S")
  print("Current thread execution time =", current_time)
  
  
  print("Current Topic: ", top_3_words_lsa)
  df = pd.DataFrame(top_3_words_lsa)
  df.to_csv(f"a_{current_time}.csv", index = False, header=True)

In [13]:
  now = datetime.now()
  current_time = now.strftime("%H_%M_%S")
  print("Current thread execution time =", current_time)

Current thread execution time = 22_30_12


In [14]:
#Scheduler
def schedule_actions():
  schedule.every().day.at('22:30:27').do(save_tiopic)
  #schedule.every(1).minutes.do(save_tiopic)
  while True:
    schedule.run_pending()
    sleep(1)

schedule_actions()

Headline before vectorization: man charge kill mother sea inherit family estate nathan carman arrive small boat coast guard station boston sept spending week sea life raft rescue michael dwyerap
Current thread execution time = 22_30_33
Current Topic:  ['abortion court supreme', 'ukraine ukrainian climate', 'new york covid', 'noose university say', 'primary ohio rep', 'press secretary jeanpierre', 'climate black year', 'oil russian sanction', 'texas abortion law', 'al kill jazeera', 'photo astronaut april', 'video sheriff game', 'gas soon export', 'covid big global', 'texas oyster mason', 'southside people school', 'drug die age', 'marcos presidential hari', 'human scientist emotion', 'solar project nasa']


KeyboardInterrupt: ignored